In [11]:
import pandas as pd
import numpy as np
import sys
import os
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

SURVEY_SUMMARY_PATH = r'E:\Met_Council\metc-asim-model\Input\Visualizer\base'
ASIM_OUTPUT = r"E:\Met_Council\metc-asim-model\Base_2018\activitysim\output"
ASIM_CONFIGS_PATH = r'E:\Met_Council\metc-asim-model\source\activitysim\configs'

DAMPING = 0.75

coefficient_dict = {
    1: {
        0: 'coef_alternative_specific_constant_for_1_driver_household_0_CARS',
        1: None,
        2: 'coef_alternative_specific_constant_for_1_driver_household_2_CARS',
        3: 'coef_alternative_specific_constant_for_1_driver_household_3_CARS',
        4: 'coef_alternative_specific_constant_for_1_driver_household_4_CARS'
    },
    2: {
        0: 'coef_alternative_specific_constant_for_2_driver_household_0_CARS',
        1: 'coef_alternative_specific_constant_for_2_driver_household_1_CAR',
        2: None,
        3: 'coef_alternative_specific_constant_for_2_driver_household_3_CARS',
        4: 'coef_alternative_specific_constant_for_2_driver_household_4_CARS'
    },
    3: {
        0: 'coef_alternative_specific_constant_for_3_driver_household_0_CARS',
        1: 'coef_alternative_specific_constant_for_3_driver_household_1_CAR',
        2: 'coef_alternative_specific_constant_for_3_driver_household_2_CARS',
        3: None,
        4: 'coef_alternative_specific_constant_for_3_driver_household_4_CARS'
    },
    4: {
        0: 'coef_alternative_specific_constant_for_4_driver_household_0_CARS',
        1: 'coef_alternative_specific_constant_for_4_driver_household_1_CAR',
        2: 'coef_alternative_specific_constant_for_4_driver_household_2_CARS',
        3: 'coef_alternative_specific_constant_for_4_driver_household_3_CARS',
        4: None
    }
}

observed_auto_ownership = pd.read_csv(r"..\target_data\pums_adults_veh.csv")
observed_auto_ownership['pct_census'] = observed_auto_ownership['num_hh'] / observed_auto_ownership['num_hh'].sum()
observed_auto_ownership = observed_auto_ownership.rename(columns = {'VEH': 'auto_ownership'}).set_index(['num_adults', 'auto_ownership'])

model_hh = pd.read_csv(os.path.join(ASIM_OUTPUT, 'final_households.csv'))
model_hh['num_adults'] = model_hh['num_adults'].clip(0, 4)
model_data = model_hh.groupby(['num_adults', 'auto_ownership']).agg(model_hh = ('household_id', 'count'))
model_data['model_pct'] = model_data['model_hh'] / model_data['model_hh'].sum()

coefficients = pd.read_csv(os.path.join(ASIM_CONFIGS_PATH, 'auto_ownership_coefficients.csv'))

In [12]:
compare_table = observed_auto_ownership.join(model_data)
compare_table['coefficient_name'] = ""
for i, v in compare_table.iterrows():
    if coefficient_dict[i[0]][i[1]] != None:
        compare_table.loc[i, 'coefficient_name'] = coefficient_dict[i[0]][i[1]]
compare_table = compare_table.merge(coefficients, how = 'left', on = 'coefficient_name')
compare_table['adj'] = np.log(compare_table['pct_census'] / compare_table['model_pct']) * DAMPING
compare_table.loc[compare_table['constrain'] != 'F', 'adj'] = 0.0
compare_table.rename(columns = {'value': 'old_value'}, inplace = True)
compare_table['value'] = compare_table['old_value'].astype(float).fillna(0) + compare_table['adj'].fillna(0)
compare_table

,num_hh,pct_census,model_hh,model_pct,coefficient_name,old_value,constrain,adj,value
0,82290,0.050537,16105,0.053683,coef_alternative_specific_constant_for_1_drive...,-2.727619,F,-0.045300,-2.772919
1,355472,0.218306,57883,0.192943,,NaN,NaN,0.000000,0.000000
2,68533,0.042088,17701,0.059003,coef_alternative_specific_constant_for_1_drive...,-1.382277,F,-0.253369,-1.635647
3,13338,0.008191,2046,0.006820,coef_alternative_specific_constant_for_1_drive...,-3.090677,F,0.137408,-2.953269
4,5168,0.003174,705,0.002350,coef_alternative_specific_constant_for_1_drive...,-2.949096,F,0.225393,-2.723704
5,20480,0.012577,4822,0.016073,coef_alternative_specific_constant_for_2_drive...,-3.297443,F,-0.183945,-3.481388
6,109869,0.067474,26898,0.089660,coef_alternative_specific_constant_for_2_drive...,-0.524986,F,-0.213212,-0.738198
7,556429,0.341720,74904,0.249680,,NaN,NaN,0.000000,0.000000
8,159103,0.097710,36090,0.120300,coef_alternative_specific_constant_for_2_drive...,-0.503439,F,-0.155988,-0.659427
9,44429,0.027285,11024,0.036747,coef_alternative_specific_constant_for_2_drive...,-1.964156,F,-0.223277,-2.187433


In [13]:
compare_table[compare_table['coefficient_name'].str.len() > 0][['coefficient_name', 'value', 'constrain']].to_csv(sys.stdout, index = False)

coefficient_name,value,constrain
coef_alternative_specific_constant_for_1_driver_household_0_CARS,-2.77291909987767,F
coef_alternative_specific_constant_for_1_driver_household_2_CARS,-1.6356466386197588,F
coef_alternative_specific_constant_for_1_driver_household_3_CARS,-2.953268666326432,F
coef_alternative_specific_constant_for_1_driver_household_4_CARS,-2.7237036457157253,F
coef_alternative_specific_constant_for_2_driver_household_0_CARS,-3.481388021407995,F
coef_alternative_specific_constant_for_2_driver_household_1_CAR,-0.738198242869457,F
coef_alternative_specific_constant_for_2_driver_household_3_CARS,-0.6594266670786043,F
coef_alternative_specific_constant_for_2_driver_household_4_CARS,-2.1874328240318937,F
coef_alternative_specific_constant_for_3_driver_household_0_CARS,-4.205190651945007,F
coef_alternative_specific_constant_for_3_driver_household_1_CAR,-2.4078395524282215,F
coef_alternative_specific_constant_for_3_driver_household_2_CARS,-1.1515200019587233,F
coef_alternative_s